In [1]:
import pandas as pd
import sys
import os
#from pprint import pprint

sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/lib")
#from db_query import connectDB, nextPrototypeId, queryPrototype, queryCosts
from db_query import connectDB, queryPrototype, queryCosts, queryPrototypeParams
from ats_file_tools import oldest_dp_file_pair, archive_file
from ats_tools import add_candidate

In [2]:
archive_dir = '/Users/szagar/ZTS/Dropbox/Business/ats/Archive/DPfiles'
F1 = 2
F2 = 4
F3 = 8
F4 = 16
F5 = 32
F6 = 64
F7 = 128
F8 = 256
F9 = 512
F10 = 1024

In [3]:
def prepare_data(dbh):
    is_file,oos_file = oldest_dp_file_pair()
    if not is_file:
        return None
    #is_file = archive_file(archive_dir,is_file)
    #oos_file = archive_file(archive_dir,oos_file)
    
    proto_id = os.path.basename(is_file).split('_')[1]
    print(f"proto_id={proto_id}")
    
    df_is = pd.read_csv(is_file,sep='\t',index_col='Test')
    df_oos = pd.read_csv(oos_file,sep='\t',index_col='Test')
        
    strat_name = [c.split(':')[0] for c in df_is.columns if c.startswith('proto_')][0]
    
    df_oos.rename(columns=lambda n: n.replace(f'{strat_name}: ', 'param:'), inplace=True)
    df_oos.rename(columns=lambda n: n.replace('All:', 'OOS:'), inplace=True)
    df_is.rename(columns=lambda n: n.replace(f'{strat_name}: ', 'param:'), inplace=True)
    df_is.rename(columns=lambda n: n.replace('All:', 'IS:'), inplace=True)
    
    df = df_is.join(df_oos.loc[:,df_oos.columns.str.startswith('OOS:')])
    df['IS: NP2DD'] = df['IS: Net Profit'] / -df['IS: Max Intraday Drawdown']
    df['OOS: NP2DD'] = df['OOS: Net Profit'] / -df['OOS: Max Intraday Drawdown']
    
    df['IS: Expectancy'] = df['IS: Avg Winning Trade']*df['IS: % Profitable']/100 - df['IS: Avg Losing Trade']*(1-df['IS: % Profitable']/100)


    
    df['OOS: Expectancy'] = df['OOS: Avg Winning Trade']*df['OOS: % Profitable']/100 - df['OOS: Avg Losing Trade']*(1-df['OOS: % Profitable']/100)

    #df2 = df[(df['IS: Net Profit']>0) &
    #         (df['OOS: Net Profit']>0) &
    #         (df['IS: Robustness Index']>60) &
    #         (df['IS: Avg Trade']>60) &
    #         (df['OOS: Total Trades']>10)].copy()    #.sort_values(by='IS: TradeStation Index',ascending=False,inplace=True)
    
    df['AvgTrade_Filter'] = 0
    df['Quartile'] = 0
    df.sort_values(['IS: TradeStation Index'], ascending=False,inplace=True)
        
    set_filters(dbh,df,proto_id)
    
    return proto_id,df

#tmp_df = prepare_data()
##if tmp_df:
#tmp_df.loc[:,['poi','natr','IS: TradeStation Index']].head(5)

In [4]:
def set_filters(dbh,df,proto_id,filter_list=(F1,F2,F3,F4,F5,F6)):
    if F1 in filter_list:
        avg_trade_level = 60
        print(f"F1 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>60) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F1
    if F2 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 2 * (slippage + commission)
        print(f"F2 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F2
    if F3 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 3 * (slippage + commission)
        print(f"F3 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F3
    if F4 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 5 * (slippage + commission)
        print(f"F4 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F4
    if F5 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 6 * (slippage + commission)
        print(f"F5 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F5
    if F6 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 7 * (slippage + commission)
        print(f"F6 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F6
    if F7 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 8 * (slippage + commission)
        print(f"F7 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F7
    if F8 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 9 * (slippage + commission)
        print(f"F8 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F8
    if F9 in filter_list:
        proto = queryPrototype(dbh,proto_id)
        #slippage, commission = queryCosts(dbh,proto[0].symbol)
        slippage, commission = queryCosts(dbh,proto.symbol)
        avg_trade_level = 10 * (slippage + commission)
        print(f"F9 avg_trade_level={avg_trade_level}")
        df.loc[(df['IS: Net Profit']>0) & 
               (df['OOS: Net Profit']>0) & 
               (df['IS: Robustness Index']>60) & 
               (df['IS: Avg Trade']>avg_trade_level) & 
               (df['OOS: Total Trades']>10), 'AvgTrade_Filter'] = df['AvgTrade_Filter'] | F9


In [5]:
def record_candidates(dbh,proto_id,params):
    #print(params)
    #dbh = connectDB()
    add_candidate(dbh,proto_id,params)


# def capture_params(dbh,proto_id,df,top_q=[]):
    filter_col = [col for col in df if col.startswith('param:')]
    cnt = 0
    for i,j in df.iterrows():
        cand_params = {}
        cand_params['Test'] = i
        hash_keys = []
        for c in filter_col:
            col_name = c.split(':')[1]
            cand_params[col_name] = j[c]
            hash_keys.append(str(j[c]))
        hk = ':'.join(hash_keys)
        if hk in _param_hash:
            #print("Found duplicate candidate")
            continue
        _param_hash[hk] = 1
        cnt += 1
        cand_params['filters'] = int(j['AvgTrade_Filter'])
        cand_params['top_q'] = ','.join(top_q)
        record_candidates(dbh,proto_id,cand_params)
    return cnt

In [7]:
def select_candidates(dbh,proto_id,df):
    cand_cnt = 0
    
    #top_q = ['OOS: Avg Trade|90','IS: Avg Trade|90']
    for f_level in (F10,F9,F8,F7,F6,F5,F4,F3,F2,F1):
        for trade_cnt in (100,90,80,70,60):
            quantile = 90
            top_q = [f'OOS: Avg Trade|{quantile}',f'IS: Avg Trade|{quantile}']
            if cand_cnt > 20:
                print(f"{top_q}: f_level: {f_level}, trade_cnt: {trade_cnt}: {cand_cnt}")
                return cand_cnt
            cand_df = df[(df['AvgTrade_Filter']&f_level==f_level) & (df['IS: Total Trades']>trade_cnt) & (df['OOS: Avg Trade'] >= df['OOS: Avg Trade'].quantile(quantile/100.0))  & (df['IS: Avg Trade'] >= df['IS: Avg Trade'].quantile(quantile/100.0))]
            cand_cnt += capture_params(dbh,proto_id,cand_df,top_q=top_q)
            print(f"{top_q}: f_level: {f_level}, trade_cnt: {trade_cnt}: {cand_cnt}")

    #top_q = ['OOS: Avg Trade|90']
    for f_level in (F10,F9,F8,F7,F6,F5,F4,F3,F2,F1):
        for trade_cnt in (100,90,80,70,60):
            quantile = 90
            top_q = [f'OOS: Avg Trade|{quantile}',]
            if cand_cnt > 20:
                print(f"{top_q}: f_level: {f_level}, trade_cnt: {trade_cnt}: {cand_cnt}")
                return cand_cnt
            cand_df = df[(df['AvgTrade_Filter']&f_level==f_level) & (df['IS: Total Trades']>trade_cnt) & (df['OOS: Avg Trade'] >= df['OOS: Avg Trade'].quantile(quantile/100.0))]
            cand_cnt += capture_params(dbh,proto_id,cand_df,top_q=top_q)
            print(f"{top_q}: f_level: {f_level}, trade_cnt: {trade_cnt}: {cand_cnt}")
        
        
        

In [8]:
_param_hash = {}
dbh = None
dbh = dbh or connectDB()
proto_id,df=prepare_data(dbh)
select_candidates(dbh,proto_id,df)

/Users/szagar/ZTS/Dropbox/Business/ats/Data/Queue/GaProtoResults/proto_2_is.csv
proto_id=2
F1 avg_trade_level=60
F2 avg_trade_level=46.00000000
F3 avg_trade_level=69.00000000
F4 avg_trade_level=115.00000000
F5 avg_trade_level=138.00000000
F6 avg_trade_level=161.00000000
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 1024, trade_cnt: 100: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 1024, trade_cnt: 90: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 1024, trade_cnt: 80: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 1024, trade_cnt: 70: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 1024, trade_cnt: 60: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 512, trade_cnt: 100: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 512, trade_cnt: 90: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 512, trade_cnt: 80: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 512, trade_cnt: 70: 0
['OOS: Avg Trade|90', 'IS: Avg Trade|90']: f_level: 512,

42

# proto_id = 1
dbh = connectDB()
proto_id,tmp_def=prepare_data(dbh)
tmp_def.loc[(tmp_def['OOS: Avg Trade'] >= tmp_def['OOS: Avg Trade'].quantile(0.5)), 'Quartile'] = 1
#set_filters(tmp_def,proto_id)
tmp_def[(tmp_def['passed_filters']&F5==F5) & (tmp_def['IS: Total Trades']>100)][['IS: Avg Trade','OOS: Avg Trade','IS: Net Profit','OOS: Net Profit','OOS: NP2DD','IS: Total Trades','IS: Required Account Size','Quartile','passed_filters',]].head(10)
#tmp_def.columns

#t = tmp_def[(tmp_def['passed_filters']&F5==F5) & (tmp_def['IS: Total Trades']>100) & (tmp_def['OOS: Avg Trade'] >= tmp_def['OOS: Avg Trade'].quantile(0.9))]
#select_candidates(proto_id,t,top_ten=['OOS: Avg Trade'])

t = tmp_def[(tmp_def['IS: Total Trades']>100) & (tmp_def['OOS: Avg Trade'] >= tmp_def['OOS: Avg Trade'].quantile(0.5)) & (tmp_def['IS: Avg Trade'] >= tmp_def['IS: Avg Trade'].quantile(0.5))]
select_candidates(proto_id,t,top_ten=['OOS: Avg Trade','IS: Avg Trade'])

In [ ]:
tmp_def[(tmp_def['passed_filters']&F5==F5) & (tmp_def['IS: Total Trades']>100) & (tmp_def['OOS: Avg Trade'] >= tmp_def['OOS: Avg Trade'].quantile(0.5))][['IS: Avg Trade','OOS: Avg Trade','IS: Net Profit','OOS: Net Profit','OOS: NP2DD','IS: Total Trades','IS: Required Account Size','Quartile','passed_filters',]].head(10)


In [ ]:
t = tmp_def[(tmp_def['passed_filters']&F5==F5) & (tmp_def['IS: Total Trades']>100) & (tmp_def['OOS: Avg Trade'] >= tmp_def['OOS: Avg Trade'].quantile(0.9))]
filter_col = [col for col in tmp_def if col.startswith('param:')]
for i,j in t.iterrows():
    cand_params = {}
    for c in filter_col:
        col_name = c.split(':')[1]
        cand_params[col_name] = j[c]
    cand_params['filters'] = int(j['passed_filters'])
    print(cand_params)

In [ ]:
tmp_def.columns

In [ ]:
tmp_def[['IS: Avg Trade','IS: Total Trades','IS: Avg Winning Trade','IS: % Profitable','IS: Avg Losing Trade']].sort_values(by='IS: Total Trades',ascending=False)


In [ ]:
tmp_def[['IS: Net Profit','IS: Max Intraday Drawdown','IS: NP2DD','OOS: NP2DD',
    'IS: TradeStation Index','OOS: TradeStation Index','IS: Expectancy']].sort_values(by='IS: TradeStation Index',ascending=False).head(5)

In [ ]:
df[['IS: Net Profit','OOS: Net Profit','IS: Max Intraday Drawdown','OOS: Max Intraday Drawdown']].head(5

In [ ]:
tmp_def.columns

In [ ]:
[IN: Net Profit]>0  AND  
[OOS: Net Profit]>0  AND  
[OOS vs IS AVG trade]>70  AND  
[ALL: Robustness Index]>60  AND  
[ALL: NP:DD ratio]>1  AND  
[IN: AVG trade]>60  AND  
[IN: Avg trades num per year]>40   AND  
[OOS: Avg trades num per year]>40   AND 
[OOS: Total Trades]>10 AND [Duplicity]<95

In [ ]:


#df.loc[:,['poi','natr','IS: TradeStation Index','passed_filters']].head(5)

In [ ]:
avg_trade_level = 2 * (slippage + commission)
df.loc[(df['IS: Net Profit']>0) & 
       (df['OOS: Net Profit']>0) & 
       (df['IS: Robustness Index']>60) & 
       (df['IS: Avg Trade']>avg_trade_level) & 
       (df['OOS: Total Trades']>10), 'passed_filters'] = df['passed_filters'] | F2

In [ ]:
df.loc[df['passed_filters']==F1,['poi','passed_filters']]